In [3]:
from gurobipy import *

# define a list of the loan types to use for variable indices
loan_type=['Personal', 'Car', 'Home', 'Farm', 'Commercial']

# use dictionaries to define parameters
interest={
        'Personal':     0.14,
        'Car':          0.13,
        'Home':         0.12,
        'Farm':         0.125,
        'Commercial':   0.1
        }

bad_debt={
        'Personal':     0.1,
        'Car':          0.07,
        'Home':         0.03,
        'Farm':         0.05,
        'Commercial':   0.02
        }

# create a model object
m = Model()

# define variables using loan_type as index (note nonegativity specified with lb=0.0)
# note that the "addVars" command has an "s" at the end and "loan_type" is a list used for variable indices
# variables represent the amount of money used for each loan type
x=m.addVars(loan_type, lb=0.0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name="x")

# define objective function using a "sum()" function
z = m.setObjective(sum(interest[i]*(1-bad_debt[i])*x[i] for i in loan_type), GRB.MAXIMIZE)

# define constraints -- these two constraints are the same...just showing you different ways to code them
m.addConstr(x['Home'] - 0.5*(x['Personal']+x['Car']+x['Home'])>= 0, "home_req't")
m.addConstr((x['Home'] - 0.5*sum(x[i] for i in ['Personal','Car','Home'])>= 0), "home_req't_v2")

# define more constraints
m.addConstr(x['Farm']+x['Commercial'] - 0.4*sum(x[i] for i in loan_type)>= 0, "farm/comm_req't")
m.addConstr((sum(bad_debt[i]*x[i] - 0.04*x[i] for i in loan_type)<=0),"bad_debt_limit")
m.addConstr((sum(x[i] for i in loan_type)<=12000000),"loan_amounts_add_to_$12M")

# m.addConstrs((x[i] <= 3000000 for i in loan_type),"no_loan_more_than_$3M")

# find optimal solution
m.optimize()

# write out model
m.write("L3E1.lp")

# print variable names and values
for v in m.getVars():
    print(v.varName, v.x)
     
# print objective function value    
print('Obj:', m.objVal)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 5 rows, 5 columns and 21 nonzeros
Model fingerprint: 0x105bebd2
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+07, 1e+07]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.8005000e+29   7.120000e+30   5.800500e-01      0s
       4    1.4113714e+06   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds
Optimal objective  1.411371429e+06
x[Personal] 342857.14285714366
x[Car] 0.0
x[Home] 6857142.857142856
x[Farm] 4800000.0
x[Commercial] 0.0
Obj: 1411371.4285714286
